In [ ]:
!pip install d2l==1.0.0-beta0

Một yếu tố đằng sau thành công của deep learning là sự sẵn có của nhiều lớp có thể được sắp xếp theo những cách sáng tạo để thiết kế kiến ​​trúc phù hợp với nhiều nhiệm vụ khác nhau. Chẳng hạn, các nhà nghiên cứu đã phát minh ra các lớp đặc biệt để xử lý hình ảnh, văn bản, lặp qua dữ liệu tuần tự và thực hiện lập trình động. Sớm hay muộn, bạn sẽ bắt gặp hoặc phát minh ra một lớp chưa tồn tại trong khuôn khổ học tập sâu. Trong những trường hợp này, bạn phải xây dựng một lớp tùy chỉnh. Trong phần này, chúng tôi chỉ cho bạn cách thực hiện.

In [2]:
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


# 6.5.1. Lớp không có tham số

Để bắt đầu, chúng tôi xây dựng một lớp tùy chỉnh không có bất kỳ tham số nào của riêng nó. Điều này có vẻ quen thuộc nếu bạn nhớ lại phần giới thiệu của chúng tôi về mô-đun trong Phần 6.1 . Lớp sau đây CenteredLayerchỉ cần trừ giá trị trung bình từ đầu vào của nó. Để xây dựng nó, chúng ta chỉ cần kế thừa từ lớp lớp cơ sở và thực hiện chức năng lan truyền về phía trước.

In [3]:
class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

Hãy xác minh rằng lớp của chúng ta hoạt động như dự định bằng cách cung cấp một số dữ liệu thông qua nó.

In [4]:
layer = CenteredLayer()
layer(torch.tensor([1.0, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

Bây giờ chúng ta có thể kết hợp lớp của mình như một thành phần trong việc xây dựng các mô hình phức tạp hơn.

In [5]:
net = nn.Sequential(nn.LazyLinear(128), CenteredLayer())

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Để kiểm tra độ chính xác bổ sung, chúng tôi có thể gửi dữ liệu ngẫu nhiên qua mạng và kiểm tra xem giá trị trung bình có thực sự bằng 0 hay không. Bởi vì chúng tôi đang xử lý các số dấu phẩy động, chúng tôi vẫn có thể thấy một số khác 0 rất nhỏ do lượng tử hóa.

In [6]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(3.7253e-09, grad_fn=<MeanBackward0>)

# 6.5.2. Lớp có tham số.

Bây giờ chúng ta đã biết cách xác định các lớp đơn giản, hãy chuyển sang xác định các lớp với các tham số có thể được điều chỉnh thông qua đào tạo. Chúng ta có thể sử dụng các hàm tích hợp để tạo tham số, cung cấp một số chức năng quản lý cơ bản. Cụ thể, chúng chi phối quyền truy cập, khởi tạo, chia sẻ, lưu và tải các tham số mô hình. Bằng cách này, trong số các lợi ích khác, chúng tôi sẽ không cần phải viết các thủ tục tuần tự hóa tùy chỉnh cho mọi lớp tùy chỉnh.

Bây giờ, hãy triển khai phiên bản lớp được kết nối đầy đủ của riêng chúng ta. Nhớ lại rằng lớp này yêu cầu hai tham số, một để biểu thị trọng số và một để thể hiện độ lệch. Trong quá trình triển khai này, chúng tôi sử dụng kích hoạt `ReLU` làm mặc định. Lớp này yêu cầu hai đối số đầu vào: `in_units` và `units`, tương ứng biểu thị số lượng đầu vào và đầu ra.

In [7]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))

    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

Tiếp theo, chúng tôi khởi tạo Class `MyLinear` và truy cập các tham số mô hình của nó.


In [8]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[ 0.2680, -0.6976,  2.0517],
        [-1.6962, -1.2101,  0.3808],
        [-2.1717, -0.1002,  0.0741],
        [ 0.3838,  1.6071,  0.4284],
        [ 0.3019,  0.7224,  1.3620]], requires_grad=True)

Chúng tôi có thể trực tiếp thực hiện các tính toán lan truyền về phía trước bằng cách sử dụng các lớp tùy chỉnh.

In [9]:
linear(torch.rand(2, 5))

tensor([[0.0000, 0.0000, 1.3159],
        [0.0000, 0.0000, 1.5154]])

Chúng tôi cũng có thể xây dựng các mô hình bằng cách sử dụng các lớp tùy chỉnh. Khi đã có, chúng ta có thể sử dụng nó giống như lớp được kết nối đầy đủ được tích hợp sẵn.

In [10]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[ 4.5901],
        [10.0833]])